Feb 21

The OWER classifier does learn something - training loss decreases
 as does validation until the 5th epoch or so - but
validation loss decreases only slightly and the results on
"Barack Obama is a married, male, American actor" look bad.

It is worth turning the weights of the loss function but still the
resulting classes for Obama differ randomly from run to run. Also,
plotting the class/word attentions shows that those converge but
the observable correlations are not as expected.

In the following a minimal example shall be constructed to proof
the concept behind the classifier. An entity with a few short
sentences from a small vocabulary shall be forwarded through a
pre-trained OWER classifier whose class embeddings perfectly match
the sentences so that the correct classes should be predicted.

Example scenario:

```
Input sentences: "married married", "male male", "american american"
Expected classes: married=True, male=True, american=True, actor=?
```